In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
def get_file_paths(exp_id):
    parent_dir = '/experiment_data'
    file_paths = FilePaths(parent_dir, exp_id, setup_directories=False)
    return file_paths

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from relnet.evaluation.file_paths import FilePaths
from relnet.objective_functions.objective_functions import LargestComponentSizeTargeted, GlobalEfficiency
from relnet.visualization import *

dims = (16.54, 6.20)
sns.set(font_scale=2.25)
plt.rcParams["lines.linewidth"] = 4
plt.rc('font', family='serif')
set_latex_if_required()

pad = 2.5

obj_funs = [GlobalEfficiency(), LargestComponentSizeTargeted()]
num_seeds = 1000


exp_id = 'ar_test'
fp = get_file_paths(exp_id)
csv_in = fp.figures_dir / f"R_data_{num_seeds}.csv"

df = pd.read_csv(csv_in, usecols=[1,2,3,4])
ns = [25, 50, 75]



In [ ]:
df.head(5)

In [ ]:
cols = ns
rows = [obj_fun.name for obj_fun in obj_funs]

fig, axes = plt.subplots(len(obj_funs), len(ns), figsize=dims, squeeze=False)
fig.tight_layout()
#plt.subplots_adjust(wspace=0.3) #, hspace=None)

for i, obj_fun in enumerate(obj_funs):
    for j, n in enumerate(ns):
        g_df = pd.DataFrame(df[(df['obj_fun'] == obj_fun.name) & (df['g_size'] == n)])
        
        R_dist_df = pd.DataFrame(g_df.groupby('seed', group_keys=False).apply(lambda df: df.sample(1)))
        R_data = np.array(R_dist_df["R"])
        
        axes[i][j].scatter(R_data, [0 for _ in range(len(R_data))], s=8, c=".1")
        sns.kdeplot(data=R_data, ax=axes[i][j])
        axes[i][j].set_ylim(bottom=-1)

        if j==0:
            axes[i][j].set_ylabel("Est. Density")
        axes[i][j].set_xlabel("R")
    
for ax, col in zip(axes[0], cols):
    ax.annotate(f"$N =$ {col}",
                xy=(0.5, 1), xytext=(0, pad),
                xycoords='axes fraction', textcoords='offset points',
                size='medium', ha='center', va='baseline')
    
for ax, row in zip(axes[:, 0], rows):
    ax.annotate(f"{objective_function_display_names[row]}", xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - pad, 0),
                rotation=90,
                xycoords=ax.yaxis.label, textcoords='offset points',
                size='medium', ha='right', va='center')
    
fig_file = fp.figures_dir / f'subsets_R_dist.pdf'
fig.savefig(fig_file, bbox_inches='tight')
plt.close()
        